## US and EU Sanctions on Russia after 2014

This notebook is part of my master's thesis on examining the impact of economic sanctions on Russia after its annexation of Crimea in 2014. It cleans the official sanctions lists of the EU and the US, divides each of them into targeted individuals and targeted entities, and then merges them to a comprehensive list of sanctioned individuals and sanctioned entities. It then identifies firms that were sanctioned indirectly (because either the majority shareholder or the global ultimate owner was sanctioned) and finally matches the comprehensive dataset of all targeted entities with data on economic performance from the ORBIS Europe dataset.

In [183]:
!pandoc --version

'pandoc' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

### 1. Cleaning the EU Sanctions List

In [2]:
#Importing the EU sanctions list
eu_sanctions = pd.read_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/eu_sanctions/eu_sanctions.csv").iloc[:, 1:]

In [4]:
#Assigning names to columns
column_names = ["full_name", "identifying_information", "reasons", "date_of_listing", "type"]
eu_sanctions.columns=column_names

In [5]:
#Adding a 'eu_sanction' column and setting it to 1
eu_sanctions["eu_sanction"] = 1

In [6]:
#Transform the 'date_of_listing' column into datetime format
eu_sanctions["date_of_listing"] = pd.to_datetime(eu_sanctions["date_of_listing"], format="%d.%m.%Y")

In [7]:
#Transforming names into upper cases
eu_sanctions["full_name"] = eu_sanctions["full_name"].str.upper()

In [8]:
#Filtering for English names by removing names in Russian which are in parentheses
eu_sanctions["full_name"]=eu_sanctions["full_name"].apply(lambda x: x.split("(")[0])

In [9]:
#Removing aliases which occur after line breaks
eu_sanctions["full_name"]=eu_sanctions["full_name"].apply(lambda x: x.split("\n")[0])

In [10]:
#Addressing an individual format error of the original EU sanctions list
eu_sanctions.loc[eu_sanctions["full_name"] == "DIMITRIY ", "full_name"] = "DIMITRIY VALERIEVICH UTKIN"

In [11]:
#Dividing sanctioned entities and individuals in separate dataframes
eu_sanctions_entities = eu_sanctions[eu_sanctions["type"]=="Entity"]
eu_sanctions_individuals = eu_sanctions[eu_sanctions["type"]=="Individual"]

In [12]:
#Saving the dataframe of sanctioned entities
eu_sanctions_entities.to_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/eu_sanctions/eu_sanctions_entities.csv")

In [17]:
#Splitting the 'full_name' column into given and last names
eu_sanctions_individuals[["given_name", "last_name_eu"]] = eu_sanctions_individuals["full_name"]\
                                                                        .str.rsplit(n=1, expand=True)

In [14]:
#Rearranging the order of columns
eu_sanctions_individuals = eu_sanctions_individuals[["full_name","given_name", "last_name_eu", "type", "date_of_listing", "eu_sanction", "identifying_information", "reasons"]]

In [15]:
#Saving the dataframe of sanctioned individuals
eu_sanctions_individuals.to_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/eu_sanctions/eu_sanctions_individuals.csv")

### 2. Cleaning the US Sanctions List

In [18]:
#Importing the US Sanctions list from the OFAC website
us_sanctions = pd.read_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/us_sanctions/us_sanctions.csv", header=None)

In [20]:
#Assigning names to columns
column_names = ["full_name", "address", "type", "program", "list", "other"]
us_sanctions.columns=column_names

In [21]:
#Dropping irrelevant columns
us_sanctions = us_sanctions.drop(columns=["list", "other"])

In [22]:
#Examining the types of sanctioned items
us_sanctions["type"].value_counts()

type
Entity        475
Individual    258
Vessel         10
Aircraft        6
Name: count, dtype: int64

In [23]:
#Removing vessels and aircrafts
relevant_types = ["Entity", "Individual"]
us_sanctions=us_sanctions[us_sanctions["type"].isin(relevant_types)]

In [24]:
#Splitting the 'program' column to only contain sanctions programs related to Ukraine
us_sanctions["program"] = us_sanctions["program"].apply(lambda x: str(x).split(";"))
us_sanctions["program"] = us_sanctions["program"]\
                            .apply(lambda x: ', '.join([value.strip() for value in x if 'UKRAINE' in value]))

In [25]:
#Examining the remaining programs
us_sanctions["program"].value_counts()

program
UKRAINE-EO13662                     300
UKRAINE-EO13661                     180
UKRAINE-EO13660                     152
UKRAINE-EO13685                      91
UKRAINE-EO13661, UKRAINE-EO13662      7
UKRAINE-EO13660, UKRAINE-EO13685      1
UKRAINE-EO13661, UKRAINE-EO13685      1
UKRAINE-EO13660, UKRAINE-EO13661      1
Name: count, dtype: int64

In [26]:
#Creating a dictionary of sanctions programs and their respective implementation date
program_dates = {"UKRAINE-EO13660":"2014-06-03",
                 "UKRAINE-EO13661":"2014-03-16",
                 "UKRAINE-EO13662":"2014-03-20",
                 "UKRAINE-EO13685":"2014-12-19",
                 "UKRAINE-EO13660, UKRAINE-EO13661":"2014-06-03",
                 "UKRAINE-EO13660, UKRAINE-EO13685":"2014-06-03",
                 "UKRAINE-EO13661, UKRAINE-EO13685":"2014-03-16",
                 "UKRAINE-EO13661, UKRAINE-EO13662":"2014-03-16"}

In [27]:
#Adding a 'date_of_listing' column containing the implementation date and converting it 
us_sanctions["date_of_listing"] = pd.to_datetime(us_sanctions["program"].map(program_dates))

In [28]:
#Adding a 'us_sanction' column and setting it to 1
us_sanctions["us_sanction"] = 1

In [29]:
#Transforming the 'name' into upper case (to match the format of the EU sanctions list)
us_sanctions["full_name"]=us_sanctions["full_name"].str.upper()

In [30]:
#Dividing sanctioned entities and individuals in separate dataframes
us_sanctions_entities = us_sanctions[us_sanctions["type"]=="Entity"]
us_sanctions_individuals = us_sanctions[us_sanctions["type"]=="Individual"]

In [31]:
#Saving the dataframe of sanctioned entities
us_sanctions_entities.to_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/us_sanctions/us_sanctions_entities.csv")

In [32]:
#Transforming the names of individuals to align with the format of the EU sanctions list
def transform_name(name):
    name_parts = name.split(", ")
    if len (name_parts)==2:
        last_name,first_name = name.split(", ")
        reordered_name = "".join(first_name)+" "+last_name
        return reordered_name
    else:
        return name
    
us_sanctions_individuals.loc[:, "full_name"] = us_sanctions_individuals["full_name"].apply(transform_name)

In [33]:
#Splitting the 'full_name' column into given and last names
us_sanctions_individuals[["given_name", "last_name_us"]] = us_sanctions_individuals["full_name"]\
                                                                                .str.rsplit(n=1, expand=True)

In [34]:
#Rearranging the order of columns
us_sanctions_individuals = us_sanctions_individuals\
    [["full_name", "given_name", "last_name_us", "type", "date_of_listing", "us_sanction", "program"]]

In [36]:
#Saving the dataframe of sanctioned individuals
us_sanctions_individuals.to_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/us_sanctions/us_sanctions_individuals.csv")

### 3. Merging the Individuals of the EU and US Sanctions List

In [37]:
#Conducting fuzzy matching on the 'last_name' columns of the EU and US sanctions list with a threshold of 80
def fuzzy_merge(df_1, df_2, key1, key2, threshold=80, limit=4):
    
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    merged_df = pd.merge(df_1, df_2, how='outer', left_on='matches', right_on=key2, suffixes=('_eu', '_us'))
    
    return merged_df

merged_individuals = fuzzy_merge(eu_sanctions_individuals, us_sanctions_individuals, "last_name_eu", 'last_name_us')

In [38]:
merged_individuals.columns

Index(['full_name_eu', 'given_name_eu', 'last_name_eu', 'type_eu',
       'date_of_listing_eu', 'eu_sanction', 'identifying_information',
       'reasons', 'matches', 'full_name_us', 'given_name_us', 'last_name_us',
       'type_us', 'date_of_listing_us', 'us_sanction', 'program'],
      dtype='object')

In [39]:
#Rearranging the columns in a way that allows for easy manual review of the matching process
matching_review =  merged_individuals[["full_name_eu", "full_name_us", "identifying_information",
                                       "reasons","date_of_listing_eu", "type_eu", "eu_sanction", "given_name_eu",
                                       "last_name_eu", "date_of_listing_us", "us_sanction", "program"]]

#Saving matching_review and checking it manually for false positives
matching_review.to_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/fuzzy_matching/matching_review.csv")

In [125]:
#Loading the manually reviewed matching result
sanctioned_individuals = pd.read_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/fuzzy_matching/fuzzy_matched_individuals.csv")

In [126]:
#Filtering and arranging the relevant columns for analysis
sanctioned_individuals = sanctioned_individuals[["full_name", "given_name_eu", "last_name_eu", "type", "date_of_listing_eu",
                                                "date_of_listing_us", "eu_sanction", "us_sanction"]]

In [127]:
#Renaming the 'given_name_eu' and 'last_name_eu' columns
sanctioned_individuals.rename(columns={"given_name_eu": "given_name",
                                       "last_name_eu": "last_name"}, inplace=True)

In [128]:
#Filling NAs in 'eu_sanction' and 'us_sanction' with 0
sanctioned_individuals[["eu_sanction", "us_sanction"]] = sanctioned_individuals[["eu_sanction", "us_sanction"]].fillna(0)

In [129]:
#Transforming the dates columns into datetime format
date_cols = ["date_of_listing_eu", "date_of_listing_us"]
sanctioned_individuals[date_cols] = sanctioned_individuals[date_cols].apply(pd.to_datetime, format="%d/%m/%Y")

In [131]:
#Saving the 'sanctioned_individuals' dataframe
sanctioned_individuals.to_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/sanction_entities_and_individuals/sanctioned_individualls.csv")

### 3. Merging the entities of EU and US Sanctions Lists

In [59]:
#Reducing legal entity types to acronyms
replace_dict = {"PUBLIC JOINT STOCK COMPANY": "PJSC",
                "JOINT-STOCK COMPANY": "J-SC", 
                "JOINT STOCK COMPANY": "JSC", 
                "LIMITED LIABILITY COMPANY": "LLC",
                "OBSHCHESTVO S OGRANICHENNOY OTVETSTVENNOSTYU":"OOO",
                "OTKRYTOE AKTSIONERNOE OBSHCHESTVO":"OAO",
                "AKTSIONERNOE OBSHCHESTVO":"AO"}

sanctions_entities_list = [us_sanctions_entities, eu_sanctions_entities]

for df in sanctions_entities_list:
    df.replace(to_replace=replace_dict, regex=True, inplace=True)

In [60]:
#Conducting a fuzzy merge on company names with a threshold of 90
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=4):
    
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    merged_df = pd.merge(df_1, df_2, how='outer', left_on='matches', right_on=key2, suffixes=('_eu', '_us'))
    
    return merged_df

merged_entities = fuzzy_merge(eu_sanctions_entities, us_sanctions_entities,"full_name", "full_name", threshold=90)

In [61]:
#Rearranging the columns in a way that allows for easy manual review of the matching process
matching_review_entities =  merged_entities[["full_name_eu", "full_name_us", "identifying_information",
                                             "reasons","date_of_listing_eu", "type_eu", "eu_sanction",
                                             "date_of_listing_us", "us_sanction", "program"]]

#Saving matching_review and checking it manually for false positives and removing non-commercial entities
#(e.g. Luhansk Republic, Donetsk Republic)
matching_review_entities.to_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/fuzzy_matching/matching_review_entities.csv")

In [72]:
#Loading the manually reviewed matching result
sanctioned_entities = pd.read_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/fuzzy_matching/fuzzy_matched_entities.csv")

In [74]:
#Filtering and arranging the relevant columns for analysis
sanctioned_entities = sanctioned_entities[["full_name", "type", "date_of_listing_eu",
                                                "date_of_listing_us", "eu_sanction", "us_sanction"]]

In [76]:
#Filling NAs in 'eu_sanction' and 'us_sanction' with 0
sanctioned_entities[["eu_sanction", "us_sanction"]] = sanctioned_entities[["eu_sanction", "us_sanction"]].fillna(0)

In [77]:
#Transforming the dates columns into datetime format
date_cols = ["date_of_listing_eu", "date_of_listing_us"]
sanctioned_entities[date_cols] = sanctioned_entities[date_cols].apply(pd.to_datetime, format="%d/%m/%Y")

In [111]:
#Saving the 'sanctioned_entities' dataframe
sanctioned_entities.to_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/sanction_entities_and_individuals/sanctioned_entities_2.csv")

### 4. Identifying entities that were sanctioned indirectly

#### 4.1 Identifying entities whose Global Ultimate Owner was sanctioned

In [135]:
#Importing ownership data from ORBIS Europe and sanctioned_individuals
orbis_ownership_data = pd.read_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/Orbis Data/orbis_ownership_data")
sanctioned_individuals = pd.read_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/sanction_entities_and_individuals/sanctioned_individuals.csv")
sanctioned_entities = pd.read_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/sanction_entities_and_individuals/sanctioned_entities.csv")

In [120]:
#Forward filling the name column of orbis ownership data
orbis_ownership_data["Company name Latin alphabet"]=orbis_ownership_data["Company name Latin alphabet"]\
                                                                                    .fillna(method="ffill")

In [140]:
#Joining the names of all sanctioned targets (both individuals and entities)
sanctioned_names = set(sanctioned_entities["full_name"]) | set(sanctioned_individuals["full_name"])

In [143]:
#Conducting a fuzzy match between sanctioned_names and the Global Ultimate Owner column with a threshold of 90
threshold = 90

def get_max_similarity_score(name, sanctioned_names):
    return max(fuzz.token_sort_ratio(name, sanctioned_name) for sanctioned_name in sanctioned_names)

def calculate_similarity(row, sanctioned_names):
    return get_max_similarity_score(row['GUO - Name'], sanctioned_names)

orbis_ownership_data['SimilarityScore_1'] = orbis_ownership_data.apply(lambda row: calculate_similarity(row, sanctioned_names), axis=1)

GUO_sanctioned = orbis_ownership_data[orbis_ownership_data['SimilarityScore_1'] >= threshold]

In [144]:
#Adding a 'guo_sanctioned' column and setting it to 1
GUO_sanctioned["guo_sanctioned"]=1

In [145]:
#Saving GUO_sanctioned and manually removing false positives
GUO_sanctioned.to_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/sanction_entities_and_individuals/GUO_sanctioned.csv")

#### 4.2 Identifying entities whose shareholders were sanctioned

In [146]:
#Conducting a fuzzy match between sanctioned_names and the Shareholder column with a threshold of 90
threshold = 90

def get_max_similarity_score(name, sanctioned_names):
    return max(fuzz.token_sort_ratio(name, sanctioned_name) for sanctioned_name in sanctioned_names)

def calculate_similarity(row, sanctioned_names):
    return get_max_similarity_score(row['Shareholder - Name'], sanctioned_names)

orbis_ownership_data['SimilarityScore_2'] = orbis_ownership_data.apply(lambda row: calculate_similarity(row, sanctioned_names), axis=1)

shareholder_sanctioned = orbis_ownership_data[orbis_ownership_data['SimilarityScore_2'] >= threshold]

In [147]:
#Adding a 'shareholder_sanctioned' column and setting it to 1
shareholder_sanctioned["shareholder_sanctioned"]=1

In [150]:
#Cleaning the 'Shareholder - Direct %' and 'Shareholder - Total %' columns
columns_to_process = ["Shareholder - Direct %\n03/2014", "Shareholder - Total %\n03/2014"]

for column in columns_to_process:
    shareholder_sanctioned[column] = (
        shareholder_sanctioned[column]
        .replace(["-", "BR", "MO", "n.a."], np.nan)
        .astype(float)
    )

In [151]:
#Identifying entities where the majority shareholder is sanctioned
for index, row in shareholder_sanctioned.iterrows():
    if row["Shareholder - Direct %\n03/2014"] >= 50:
        shareholder_sanctioned.at[index, "maj_shareholder_sanctioned"] = 1
    elif row["Shareholder - Total %\n03/2014"] >= 50:
        shareholder_sanctioned.at[index, "maj_shareholder_sanctioned"] = 1
    else:
        shareholder_sanctioned.at[index, "maj_shareholder_sanctioned"] = 0

In [152]:
#Saving shareholder_sanctioned
shareholder_sanctioned.to_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/sanction_entities_and_individuals/shareholder_sanctioned.csv")

### 5. Merging sanctioned_entities, GUO_sanctioned, and shareholder_sanctioned

In [153]:
#Merging directly and indirectly sanctioned entities
all_sanctioned_entities = pd.concat([sanctioned_entities, GUO_sanctioned, shareholder_sanctioned],  axis=0, ignore_index=True)

In [170]:
#Replacing acronyms with legal entity names again in order to find them in the ORBIS dataset
replace_dict = {"PUBLIC JOINT STOCK COMPANY": "PJSC",
               "JOINT-STOCK COMPANY": "J-SC", 
                "JOINT STOCK COMPANY": "JSC", 
               "LIMITED LIABILITY COMPANY": "LLC",
               "OBSHCHESTVO S OGRANICHENNOY OTVETSTVENNOSTYU":"OOO",
               "OTKRYTOE AKTSIONERNOE OBSHCHESTVO":"OAO",
               "AKTSIONERNOE OBSHCHESTVO":"AO"}

back_to_original = {value: key for key, value in replace_dict.items()}

all_sanctioned_entities.replace(to_replace=back_to_original, regex=True, inplace=True)

In [171]:
#Saving all_sanctioned_entities and manually fixing any differences to names in the ORBIS dataset
all_sanctioned_entities.to_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/sanction_entities_and_individuals/all_sanctioned_entities.csv")

### 6. Matching all_sanctioned_entities with economic performance data from ORBIS Europe

In [175]:
#Loading performance data from ORBIS Europe and the manually reviewed version of all_sanctioned_entities
orbis_performance_data = pd.read_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/Orbis Data/orbis_performance_data.csv")
all_sanctioned_entities = pd.read_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/final/sanctioned_entities.csv")

In [178]:
#Filtering for sanctioned firms in the orbis_performance dataframe
sanctioned_performance_data = orbis_performance_data[orbis_performance_data["Name"].isin(all_sanctioned_entities["orbis_name"])]

In [181]:
#Save as final dataset
sanctioned_performance_data.to_csv("C:/Users/lisa/OneDrive - Nexus365/Thesis Data/final/sanctioned_performance_data_2.csv")